In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [52]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision.models import resnet50
from pathlib import Path
from PIL import Image
import numpy as np
import time
import os

In [53]:
# Path to the JPEGImages folder
data_dir = Path('/content/drive/MyDrive/JPEGImages')

# Data loading and preprocessing
class AnimalDataset(Dataset):
    def __init__(self, root_dir, train=True):
        self.root_dir = Path(root_dir)
        self.train = train
        self.classes = [d.name for d in self.root_dir.iterdir() if d.is_dir()]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.samples = [(file, self.class_to_idx[file.parent.name])
                        for cls in self.root_dir.iterdir() if cls.is_dir()
                        for file in cls.iterdir() if file.is_file() and self.is_image_file(file.name)]

        # Define transforms based on whether it's training data
        if self.train:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(15),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

    def is_image_file(self, filename):
        """Check if a file is an image."""
        image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}
        return any(filename.lower().endswith(ext) for ext in image_extensions)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, target = self.samples[idx]
        try:
            image = Image.open(path).convert('RGB')
            if self.transform:
                image = self.transform(image)
        except Exception as e:
            print(f"Error opening image: {path}, Error: {e}")
            # Optionally, handle corrupted files or read next file
            return None  # Depending on your preference
        return image, target


In [54]:
# Creating instances for training and testing datasets
train_dataset = AnimalDataset(data_dir, train=True)
test_dataset = AnimalDataset(data_dir, train=False)

# Class weights and sampler setup
class_counts = torch.zeros(len(train_dataset.classes))
for _, index in train_dataset.samples:
    class_counts[index] += 1
class_weights = 1.0 / (class_counts + 1e-6)
sample_weights = class_weights[[sample[1] for sample in train_dataset.samples]]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

# DataLoader setup
train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)


In [55]:
# Model definition
model = resnet50(pretrained=True)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(model.fc.in_features, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(512, 50)
)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Mixed precision
scaler = torch.cuda.amp.GradScaler()

In [56]:
# Training
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    start_time = time.time()  # Start time measurement
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        if i % 20 == 0:
            print(f'Epoch {epoch+1}, Step {i}, Loss: {loss.item()}')
    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f'Epoch {epoch+1} completed in {elapsed_time:.2f} seconds')





Epoch 1, Step 0, Loss: 3.999176025390625
Epoch 1, Step 20, Loss: 2.70925235748291
Epoch 1, Step 40, Loss: 2.405168294906616
Epoch 1, Step 60, Loss: 2.172438859939575
Epoch 1, Step 80, Loss: 2.2899551391601562
Epoch 1, Step 100, Loss: 1.7975573539733887
Epoch 1, Step 120, Loss: 1.865185260772705
Epoch 1, Step 140, Loss: 1.5738177299499512
Epoch 1, Step 160, Loss: 1.783432960510254
Epoch 1 completed in 37.92 seconds
Epoch 2, Step 0, Loss: 1.4050970077514648
Epoch 2, Step 20, Loss: 1.669276475906372
Epoch 2, Step 40, Loss: 1.6496264934539795
Epoch 2, Step 60, Loss: 1.626551866531372
Epoch 2, Step 80, Loss: 1.5257282257080078
Epoch 2, Step 100, Loss: 1.2218142747879028
Epoch 2, Step 120, Loss: 1.2127766609191895
Epoch 2, Step 140, Loss: 1.125311017036438
Epoch 2, Step 160, Loss: 1.0908727645874023
Epoch 2 completed in 37.39 seconds
Epoch 3, Step 0, Loss: 1.372619390487671
Epoch 3, Step 20, Loss: 0.9384704232215881
Epoch 3, Step 40, Loss: 1.3091274499893188
Epoch 3, Step 60, Loss: 1.5328093

In [57]:
# Evaluation
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Accuracy: 87.62996793314547%
